In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask
import secrets
from flask import render_template, request, redirect, flash

In [2]:
df = pd.read_excel('Recommender_Dataset.xlsx')
#df

In [3]:
def finalList(df):
    y = df.drop(columns = ['CAO opening rank','CAO closing rank','Budget','City','Interest','Job domain'])
    X = df.drop(columns = ['University', 'Course Name'])
    cityle = LabelEncoder()
    X['City']=cityle.fit_transform(X['City'])
    intle =  LabelEncoder()
    X['Interest']=intle.fit_transform(X['Interest'])
    jdle = LabelEncoder()
    X['Job domain']=jdle.fit_transform(X['Job domain'])
    ule = LabelEncoder()
    cle = LabelEncoder()
    y['University'] = ule.fit_transform(y['University'])
    y['Course Name'] = cle.fit_transform(y['Course Name'])
    return X,y,ule,cle,cityle,intle,jdle

In [4]:
def similarCourses(query,X,y,ule,cle):
    print(query)
    X = X.drop(columns = ['Budget','City','Interest','Job domain'])
    dfDense = cosine_similarity(X,query)
    sortedCS = np.zeros(len(dfDense), dtype = 'float')
    
    for i in range(0,len(dfDense)):
        sortedCS[i] = dfDense[i][0]
    
    sortedCS = np.sort(sortedCS)
    sortedCS = np.flip(sortedCS)
    
    topTen = sortedCS[0:10]
    
    topTenRec = []
    

    for i in range(0,len(dfDense)):
        for j in range(0,len(topTen)):
            if topTen[j] == dfDense[i]:
                topTenRec.append(j)
    
    result = pd.DataFrame(columns = ['courses','universities'])
    result['courses'] = cle.inverse_transform(y['Course Name'][topTenRec])
    result['universities'] = ule.inverse_transform(y['University'][topTenRec])
    #result['budget']  = X['Budget'][topTenRec]
    return result

In [5]:
app = Flask(__name__)
@app.route("/", methods=["GET","POST"])
def index():

    secret_key=secrets.token_hex(16)
    app.config["SECRET_KEY"] = secret_key
    budget = [2000,4000,8000]
    df = pd.read_excel('Recommender_Dataset.xlsx')
    X,y,ule,cle,cityle,intle,jdle = finalList(df)
    
    if(request.method == "POST"):
        req = request.form

        percentile = req["percentile"]
        rank = req["rank"]
        state = req["state"]
        pwd = req["pwd"]
        gender = req["gender"]
        category = req["category"]
        sortby = str(req["sortby"])
        print(state)
        query = np.array([rank,625]).reshape(1,-1)
        result = similarCourses(query,X,y,ule,cle)
        
        return render_template("public/result.html",tables=[result.to_html(classes='data')], titles=result.columns.values)

    return render_template("public/index.html")

In [ ]:
if __name__ == '__main__':
    
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Nov/2022 13:07:22] "POST / HTTP/1.1" 200 -


Dundalk
[['601' '625']]
